In [71]:
!pip install nltk

In [72]:
#import desired libararies
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from bs4 import BeautifulSoup

In [73]:
#import data
#read data from directory
#distinguish data
#review and sentiment
#you can copy the path by copy path option
data=pd.read_csv('../data/interim/data.csv')
df=data.copy()
#first 5 rows
df.head()
#last 5 rows
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [74]:
df.shape

(50000, 2)

In [75]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [76]:
#there is no missing values
#summition format
df.isnull().sum()
#bool format
df.isnull().any()

review       False
sentiment    False
dtype: bool

In [77]:
#note that two columns of df are numerical
df.describe().T

,count,unique,top,freq
review,50000,49582,Loved today's show!!! It was a variety and not...,5
sentiment,50000,2,positive,25000


In [78]:
for i in df.columns :
  print(f' for {i} the unique values are : ')
  print()
  print(df[i].value_counts())
  print()

 for review the unique values are : 

review
Loved today's show!!! It was a variety and not solely cooking (which would have been great too). Very stimulating and captivating, always keeping the viewer peeking around the corner to see what was coming up next. She is as down to earth and as personable as you get, like one of us which made the show all the more enjoyable. Special guests, who are friends as well made for a nice surprise too. Loved the 'first' theme and that the audience was invited to play along too. I must admit I was shocked to see her come in under her time limits on a few things, but she did it and by golly I'll be writing those recipes down. Saving time in the kitchen means more time with family. Those who haven't tuned in yet, find out what channel and the time, I assure you that you won't be disappointed.                                                                                                                                                                   

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


**First Pre Processing of data**

**Tokenization or String Operation**

In [80]:
#prepare stop words , tokens and html parsing
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:
#parse html code and erase special pattern using beautiful soup and regex
#beautiful soup is not only for web scrapping but aslo for html parsing basicly
def noise_removal(text) :
  soup=BeautifulSoup(text,"html.parser")
  text=soup.get_text()
  #remove special charcters
  text=re.sub('\[[^]]*\]','',text)
  return text

In [82]:
#apply removal function on review column
df['review']=df['review'].apply(noise_removal)

C:\Users\hp\AppData\Local\Temp\ipykernel_14764\1009416227.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup=BeautifulSoup(text,"html.parser")


In [83]:
#data after cleaning html parsers and speacial characters
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


**Stemming**

In [84]:
#apply function to return stem every word for every instace
def stemmer(text):
  ps=nltk.porter.PorterStemmer()
  #convert list to text again using .join
  text=' '.join([ps.stem(word) for word in text.split()])
  return text

In [85]:
#apply stemmer function on review column
df['review']=df['review'].apply(stemmer)

In [86]:
#data after applying stemming
df.head()

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl production. the film techniqu i...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there' a famili where a littl boy (jake)...,negative
4,"petter mattei' ""love in the time of money"" is ...",positive


**Stop Words**

In [87]:
#removing stop words
stopwords

<WordListCorpusReader in 'C:\\Users\\hp\\AppData\\Roaming\\nltk_data\\corpora\\stopwords'>

In [88]:
#tokenization is important for removing stop words later as comparing them with list of stop words later
#create tokenizer object
token=ToktokTokenizer()
#stop words in english
stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(text , is_lower_case = False) :
  #text maybe small or capital
  tokens=token.tokenize(text)
  #remove any white blank for every elemnt for list of words using list comprehensive
  tokens=[i.strip() for i in tokens]
  if is_lower_case :
    new_tokens = [i for i in  tokens if i not in stopwords]
  else :
    new_tokens = [i for i in tokens if i.lower() not in stopwords]
  #return filtered tokens converting list to string
  filtered_tokens=' '.join(new_tokens)
  return filtered_tokens

In [89]:
#apply remove stop words function on review column
df['review']=df['review'].apply(remove_stopwords)

In [90]:
#data after applying stop words removing
df.head()

,review,sentiment
0,one review ha mention watch 1 oz episod ' hook...,positive
1,wonder littl production. film techniqu veri un...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic ' famili littl boy ( jake ) think ' zomb...,negative
4,"petter mattei ' "" love time money "" visual stu...",positive


**Encoding**

In [91]:
#convert data to numerical values
#apply it for sentiment
#labeling the sentient data
#it looks like label encoder ---> it is relevant because we have 2 ids in the target column aslo it retrive us data frame
#label=LabelBinarizer()
#transformed sentiment data
#sentiment_data=label.fit_transform(df['sentiment'])
#df['sentiment']=sentiment_data[:,0]
#print(sentiment_data.shape)

**Train and Test split**

In [92]:
from sklearn.model_selection import train_test_split
#split data to predictor and target
x=df['review']
y=df['sentiment']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [93]:
x_train

39087    ' kept ask dure mani fights , scream matches ,...
30893    watch entir movie. could watch entir movie. st...
45278    touch love stori reminisc in mood love ' . dr...
16398    thi latter-day fulci schlocker total abysm con...
13653    first , firmli believ norwegian movi continu g...
                               ...                        
11284    ` shadow magic ' recaptur joy amaz first movi ...
44732    found thi movi quit enjoy fairli entertaining....
38158    avoid thi one ! terribl movie. veri exciting ?...
860      thi product wa quit surpris me. absolut love o...
15795    thi decent movie. although littl bit short tim...
Name: review, Length: 40000, dtype: object

In [94]:
x_test

33553    realli like thi summerslam due look arena , cu...
9427     mani televis show appeal quit mani differ kind...
199      film quickli get major chase scene ever increa...
12447    jane austen would definit approv thi one ! gwy...
39489    expect somewhat high went see thi movie , thou...
                               ...                        
28567    although casper van dien michael rooker gener ...
25079    like thi movie. ' realli sure wa befor start w...
18707    ye non-singaporean ' ' see ' big deal thi film...
15200    far film go , thi likabl enough. entertain cha...
5857     saw anatomi year ago -- dub friend hous ' reme...
Name: review, Length: 10000, dtype: object

In [95]:
y_train

39087    negative
30893    negative
45278    positive
16398    negative
13653    negative
           ...   
11284    positive
44732    positive
38158    negative
860      positive
15795    positive
Name: sentiment, Length: 40000, dtype: object

In [96]:
y_test

33553    positive
9427     positive
199      negative
12447    positive
39489    negative
           ...   
28567    negative
25079    positive
18707    positive
15200    negative
5857     positive
Name: sentiment, Length: 10000, dtype: object

**Bags of Words**

In [97]:
#count vectorized for bag of words
cv=CountVectorizer(min_df=0.0,max_df=1.0,binary=False,ngram_range=(1,3))
#fit then transform train data
cv_train=cv.fit_transform(x_train)
#transform test data
cv_test=cv.transform(x_test)
print('BOW for training data shape : ',cv_train.shape)
print('BOW for test data shape : ',cv_test.shape)

BOW for training data shape :  (40000, 7159691)
BOW for test data shape :  (10000, 7159691)


**TF_IDF**

In [98]:
#TFIDF vectorizer
tf=TfidfVectorizer(min_df=0.0,max_df=1.0,use_idf=True,ngram_range=(1,3))
#fit then transform train data
tf_train=tf.fit_transform(x_train)
#transform test data
tf_test=tf.transform(x_test)
print('TFIDF for training data shape : ',tf_train.shape)
print('TFIDF for training data shape : ',tf_test.shape)

TFIDF for training data shape :  (40000, 7159691)
TFIDF for training data shape :  (10000, 7159691)


In [99]:
#applying regression model
#create logistic regression object
model=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#fit it for bag of words
lr_bow=model.fit(cv_train,y_train)
print(lr_bow)
#fit it for TFIDF of words
lr_tf=model.fit(tf_train,y_train)
print(lr_tf)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


In [100]:
#predict it for bag of words
lr_bow_predict=model.predict(cv_test)
print(lr_bow_predict)
#predict it for TFIDF
lr_tf_predict=model.predict(tf_test)
print(lr_tf_predict)

['negative' 'positive' 'negative' ... 'positive' 'negative' 'positive']
['negative' 'positive' 'negative' ... 'positive' 'negative' 'positive']


In [101]:
#accuracy for bag of words
lr_bow_acc=accuracy_score(y_test,lr_bow_predict)
print(np.around(lr_bow_acc,2))
#accuracy for TFIDF
lr_tf_acc=accuracy_score(y_test,lr_tf_predict)
print(np.around(lr_tf_acc,2))

0.84
0.88


In [102]:
#save model for tfidf and another one for logistic regression and reuse it again
#we can use bag of words or TFIDF they gave the same result
import pickle
pickle.dump(tf, open('model_tfidf.pkl', 'wb'))
pickle.dump(model, open('model_log.pkl', 'wb'))